# `Практикум по программированию на языке Python`
## `Занятие 01: Оборачиваем модель в сервис. Первая часть: Docker, Flask`

### `Находнов Максим (nakhodnov17@gmail.com)`
#### `Москва, 2022`

О чём можно узнать из этого ноутбука:

* Принципы переносимой разработки приложений
* Контейнеризация
* Docker

### `Какую проблему пытаемся решить?`

* Многообразие **средств разработки** и **сред выполнения** требует сложного управления зависимостями
* Настройка окружения для комплексных проектов становиться сложной задачей
* Ручное управление всеми компонентами **непереносимо** и **трудноподдерживаемо**
* Изменение компонентов в существующей системе становиться невозможным (**dependency hell**)

![Типичный пример](https://pointful.github.io/docker-intro/docker-img/the-challenge.png)

### `Подход к решению проблемы`

Для решения такой проблемы **нужен способ абстракции**, который позволит работать сразными типами окружений и приложении схожим образом

#### `Решение из "реального мира"`

![Аналогия из жизни](https://pointful.github.io/docker-intro/docker-img/cargo-transport-pre-1960.png)

![Решение](https://pointful.github.io/docker-intro/docker-img/intermodal-shipping-container.png)

#### `Возвращаясь к програмному обеспечению`

![ISO-Docker-контейнер](https://pointful.github.io/docker-intro/docker-img/shipping-container-for-code.png)

### `Преимущества Docker контейнеров`

* **Инкапсуляция** компонентов инфраструктуры
* **Платформонезависимость** (на аппаратном и програмном уровнях)
* Инфраструктура, как **код** (версионирование, снепшоты, воспроизводимость)
* **Изоляция** кода, окружений, сред выполнения
* **Автоматизация** рутинных операций (тестирование, развёртывание, CI/CD)
* Упрощение построения **отказоустойчивых** сервисов
* **Упрощение** взаимодействия между разными командами разработчиков
* Упрощение работы по **интреграции ПО** сторонними разработчиками

И всё это с минимальными накладными расходами ресурсов!

### `Что под капотом?`

* Docker использует **ядро host** операционной системы
* За счёт **разделения ресурсов** позволяет создавать **изолированные** группы процессов и файловые системы

### `Отличия от VM / Hypervisor`

- [x] Docker не запускает отдельное ядро для каждого процесса по отдельности
- [x] Контейнеры значительно быстрее запускаются
- [x] VM требует больше ресурсов для запуска
- [x] В VM сложно контролировать изменения, версионирование
- [x] Контейнеры могут иметь разделяемые ресурсы между собой (бинарники, библиотеки)
- [x] При изменении ~контейнера~образа будет сохраняться только разница между исходником и новой версией


- [ ] VM имеют более надёжную изоляцию между собой
- [ ] VM чуть проще в использовании

![](https://pointful.github.io/docker-intro/docker-img/containers-vs-vms.png)

### `Базовая инфраструктура Docker`

![](https://pointful.github.io/docker-intro/docker-img/basics-of-docker-system.png)

![](https://pointful.github.io/docker-intro/docker-img/changes-and-updates.png)

### `Основные концепты Docker`

~Контейнеры~Образы Docker хранятся в виде **файловой системы**. Только в момент запуска происходит создание пространства процессов. При этом процессы будет работать через **ядро host системы**!

<dl>
  <dd>0. Dockerfile — <b>описание процедуры</b> создания файловой системы контейнера</dd>
  <dd>1. Layer (<b>слой</b>) — атомарный набор изменений <b>файловой системы</b></dd>
  <dd>2. Image (<b>образ</b>) — <b>RO</b> файловая система</dd>
  <dd>3. Container (<b>контейнер</b>) — образ с <b>RW</b> слоем поверх него</dd>
  <dd>3*. Running container (<b>запущенный контейнер</b>) — контейнер (RW файловая система) с <b>пространстом процессов</b></dd>
</dl>

* Слой описывается командой в Dockerfile 
* Образ состоит из слоёв
* Контейнер получен из образа
* Запущенный контейнер получен из контейнера

![](https://cdn.buttercms.com/CLQJN3yRRcS7oGqm7yKb)

### `Описание процесса создания Docker образа`

1. Взять пустую файловую систему
2. Последовательно, для каждой команды в Dockerfile
    - 1. **Добавить** поверх RW **новый слой**
    - 2. Выполнить команду и **записать изменения**, вызванные ей, **в текущий слой**
    - 3. Сделать текущий слой RO
3. Сделать последний слой RO
4. Сохранить полученный набор слоёв

### `Docker Livecycle`

![](./Docker%20Livecycle.svg)

### `Демонстрация`

#### `1. Простейший пример. Docker-hub`

```bash
docker pull hello-world
docker image ls
docker run hello-world
docker container ls -a
docker rm "<CONTAINER_ID>"
```

#### `2. Docker-hub. Ubuntu`

* Первый контейнер, который можно применять на практике
* Рассказ про выполнение команд внутри контейнера (на примере `ls`)
* Обзор флагов запуска (`--rm`, `-i`, `-t`, `-d`, `-p`, `-v`).

```bash
docker pull ubuntu
docker image ls
docker run ubuntu ls
docker rm "<CONTAINER_ID>"
# Запуск с удалением по завершению
docker run --rm ubuntu ls
# Запуск с подключением к псевдоконсоли
docker run -i -t ubuntu bash
```

#### `3.`
* Демонстрация изменений временной файловой системы внутри контейнера
* Уничтожение пространства процесса

 ```bash
 # Запуск с комбинацией -i -t позволяет выполнять detach комбинацией ^P^Q внутри контейнера
 docker run -i -t ubuntu bash
 >> echo "Hello World" > ~/test.txt
 >> cat ~/test.txt
 # Если был указан флаг --rm то контейнер будет удалён вместе с его файловой системой
 # При отсутствии этого флага уничтожается только пространство процессов, а файловая система остаётся нетронутой
 >> exit
 docker ps -a
 docker start -i "<CONTAINER_ID>"
 # Видно, что состояние файловой системы не поменялось
 >> cat ~/test.txt
 ```

#### `4.`
* Демонстрация работы с пространством процессов
* Отсоединение от контейнера
* Фоновая работа

```bash
docker run -i -t ubuntu bash
>> apt update && apt install -y tmux
# Создаём фоновое приложение
>> tmux new -s run
>> while true; do echo >> test.txt; sleep 1; done;
>> ^B D
>> exit
docker start -i "<CONTAINER_ID>"
# Видим, что выход из контейнера таким образом действительно уничтожает пространство процессов
>> tmux ls
# Создаём фоновое приложение ещё раз
>> tmux new -s run
>> while true; do echo "1" >> test.txt; sleep 1; done;
>> ^B D
# Отсоединяемся от контейнера
>> ^P ^Q
docker start -i "<CONTAINER_ID>"
# Видим, что пространство процессов осталось нетронутым
>> tmux ls
>> exit

# Для запуска в фоне можно использовать флаг -d
docker run --rm -d ubuntu bash -c "while true; do echo '0'; sleep 1; done;"
```

#### `5. Сборка контейнеров из DockerFile`

In [1]:
from IPython.display import Code
Code('./example.py', language='python')

import os


if __name__ == '__main__':
    if os.path.exists('/root/data/data.txt'):
        with open('/root/data/data.txt', 'r', encoding='utf-8') as infile:
            result = sum((float(line.strip()) for line in infile))
            print('Sum is equal to : {0:.3f}'.format(result))
    else:
        print('File /root/data/data.txt does not exist')

    print('SECRET_KEY env variable is equal to: {0}'.format(os.environ.get('SECRET_KEY', '')))

In [2]:
! wsl echo $'1.234\n2.345\n3.4314' > data.txt
! wsl cat data.txt

1.234
2.345
3.4314


In [3]:
Code('./Dockerfile', language='Dockerfile')

FROM python:3.11-slim

RUN mkdir /root/app
WORKDIR /root/app

COPY ./example.py ./
RUN chmod +x example.py

ENV SECRET_KEY hello

CMD ["python3", "example.py"]

```bash
# Сборка контейнера
docker build -t maksim64/test_app .

# Запуск контейнера
docker run maksim64/test_app

# Запуск с переменными окружения
docker run -e SECRET_KEY:hi maksim64/test_app
    
# Запуск с монтированием директорий
docker run -e SECRET_KEY=hi -v "./:/root/data" maksim64/test_app
```

### `Шпаргалка по основным командам Docker`

```bash
# Скачать контейнер из репозитория
docker pull container_name
```

```bash
# Запустить контейнер
docker run \
    [-d] [-i] [-t] [-p 1234:5000] [-v local_path:container_path] [-w container_working_path] container_name [COMMAND]
# Здесь
# -d -- запуск в фоновом режиме
# -i -- запуск в интерактивном режиме, т.е. даёт возможность взаимодействовать с контейнером через ввод
# -t -- создать псевдокоммандную строку
# -p -- пробросить порт контейнера 5000 на локальный порт 1234
# -v -- примонтировать локальный файл/папку local_path внутрь контейнера по пути container_path
# -w -- установить рабочую директорию внутри контейнера
# container_name -- имя контейнера
# COMMAND -- команда, которую нужно запустить внутри контейнера

# Например:
# docker run hello-world       # запускает команду по умолчанию внутри контейнера hello-world
# docker -i -t ubuntu bash     # запускает командную строку bash внутри контейнера ubuntu выполняя её связывание
#                                с "локальной" командой строкой (-t) и позволяя ввод в командную строку (-i) 
```

```bash
# Посмотреть список запущенных [или бывших запущенными] контейнеров
docker ps [-a]
```

```bash
# Посмотреть список процессов внутри работающего контейнера
docker top CONTAINER_ID
```

```bash
# Остановить контейнер (т.е. отправить ему SIGTERM и дать время завершиться корректно)
docker stop CONTAINER_ID
```

```bash
# Убить контейнер (т.е. отправить ему SIGKILL и завершить сразу)
docker kill CONTAINER_ID
```

```bash
# Удалить контейнер (-f удалить работающий)
docker rm [-f] CONTAINER_ID
```

```bash
# Остановить все контейнеры и удалить их
docker stop $(docker ps -a -q) && docker rm $(docker ps -a -q)
```

```bash
# Создать образ в репозитории repo_name c названием image_name и тегом image_tag (обычно latest)
docker commit -m "message" CONTAINER_ID repo_name/image_name:image_tag
```

```bash
# Посмотреть список образов [включая промежуточные образы]
docker images [-a]
```

```bash
# Удалить образ
docker image rm IMAGE_ID
```

```bash
# Собрать образ из докерфайла (если в локальной директории лежит Dockerfile)
docker build [--no-cache] -t repo_name/image_name:image_tag .
```

```bash
# Залить в публичный репозиторий hub.docker.com образ (repo_name должно совпадать с названием вашего репозитория)
docker push repo_name/image_name:image_tag
```